## Preprocessing - same as original

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=["EIN", "NAME"], inplace=True)

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ["T9", "T13", "T12", "T2", "T25", "T14", "T29", "T15", "T17"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
counts = application_df.groupby("CLASSIFICATION").count()
low_counts = counts.loc[counts["STATUS"] < 1883]
low_counts.reset_index(inplace=True)
classifications_to_replace = list(low_counts["CLASSIFICATION"])

# print(classifications_to_replace)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# # Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
application_df.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [8]:
# Convert categorical data to numeric with `pd.get_dummies`
for column in ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION',\
              'USE_CASE', 'ORGANIZATION', 'SPECIAL_CONSIDERATIONS', 'INCOME_AMT']:
        dummies = pd.get_dummies(application_df[column])
        application_df[dummies.columns] = dummies
        
application_df.drop(columns=['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION',\
              'USE_CASE', 'ORGANIZATION', 'SPECIAL_CONSIDERATIONS', 'INCOME_AMT'], inplace=True)

application_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,Other,T10,T19,T3,T4,T5,T6,...,Y,0,1-9999,10000-24999,100000-499999,10M-50M,1M-5M,25000-99999,50M+,5M-10M
0,1,5000,1,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0


In [9]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns=["IS_SUCCESSFUL"])
y = application_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate Model - with third hidden layer & more neurons

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=120, activation="relu", input_dim=40))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=90, activation="relu"))

# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=60, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 120)               4920      
                                                                 
 dense_1 (Dense)             (None, 90)                10890     
                                                                 
 dense_2 (Dense)             (None, 60)                5460      
                                                                 
 dense_3 (Dense)             (None, 1)                 61        
                                                                 
Total params: 21331 (83.32 KB)
Trainable params: 21331 (83.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

## Compile, Train and Evaluate Model - with fourth hidden layer & more neurons

In [14]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5694 - accuracy: 0.7223
Epoch 2/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5566 - accuracy: 0.7296
Epoch 3/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5528 - accuracy: 0.7293
Epoch 4/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5515 - accuracy: 0.7300
Epoch 5/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5495 - accuracy: 0.7317
Epoch 6/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5481 - accuracy: 0.7323
Epoch 7/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5472 - accuracy: 0.7335
Epoch 8/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5467 - accuracy: 0.7333
Epoch 9/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5461 - accuracy: 0.7337
Epoch 10/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5453 - accura

804/804 [==============================] - 2s 2ms/step - loss: 0.5355 - accuracy: 0.7397
Epoch 82/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5328 - accuracy: 0.7394
Epoch 83/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5331 - accuracy: 0.7395
Epoch 84/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5330 - accuracy: 0.7403
Epoch 85/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5334 - accuracy: 0.7395
Epoch 86/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5330 - accuracy: 0.7406
Epoch 87/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5331 - accuracy: 0.7403
Epoch 88/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5329 - accuracy: 0.7402
Epoch 89/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5331 - accuracy: 0.7399
Epoch 90/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5331 - accuracy: 

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5866 - accuracy: 0.7306 - 457ms/epoch - 2ms/step
Loss: 0.5865510702133179, Accuracy: 0.7306122183799744


In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_model_1 = tf.keras.models.Sequential()

# First hidden layer
nn_model_1.add(tf.keras.layers.Dense(units=200, activation="relu", input_dim=40))

# Second hidden layer
nn_model_1.add(tf.keras.layers.Dense(units=100, activation="relu"))

# Third hidden layer
nn_model_1.add(tf.keras.layers.Dense(units=60, activation="relu"))

# Fourth hidden layer
nn_model_1.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn_model_1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model_1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 200)               8200      
                                                                 
 dense_5 (Dense)             (None, 100)               20100     
                                                                 
 dense_6 (Dense)             (None, 60)                6060      
                                                                 
 dense_7 (Dense)             (None, 30)                1830      
                                                                 
 dense_8 (Dense)             (None, 1)                 31        
                                                                 
Total params: 36221 (141.49 KB)
Trainable params: 36221 (141.49 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
# Compile the model
nn_model_1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Train the model
fit_model_1 = nn_model.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5317 - accuracy: 0.7407
Epoch 2/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5329 - accuracy: 0.7406
Epoch 3/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5311 - accuracy: 0.7413
Epoch 4/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5310 - accuracy: 0.7415
Epoch 5/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5313 - accuracy: 0.7420
Epoch 6/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5315 - accuracy: 0.7411
Epoch 7/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5307 - accuracy: 0.7409
Epoch 8/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5308 - accuracy: 0.7413
Epoch 9/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5314 - accuracy: 0.7409
Epoch 10/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5308 - accura

804/804 [==============================] - 2s 2ms/step - loss: 0.5299 - accuracy: 0.7407
Epoch 82/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5298 - accuracy: 0.7411
Epoch 83/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5299 - accuracy: 0.7418
Epoch 84/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5302 - accuracy: 0.7419
Epoch 85/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5297 - accuracy: 0.7416
Epoch 86/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5317 - accuracy: 0.7416
Epoch 87/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5296 - accuracy: 0.7419
Epoch 88/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5290 - accuracy: 0.7420
Epoch 89/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5301 - accuracy: 0.7423
Epoch 90/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5298 - accuracy: 

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model_1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6993 - accuracy: 0.5356 - 550ms/epoch - 2ms/step
Loss: 0.6993111371994019, Accuracy: 0.5356268286705017


## Re-Preprocessing - drop Spacial Considerations column and increase number of Classification bins

In [53]:
# Read in the original file for reprocessing
application_df_2 = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df_2.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [54]:
# Drop the non-beneficial ID columns, 'EIN', 'NAME', "SPECIAL_CONSIDERATIONS"
application_df_2.drop(columns=["EIN", "NAME", "SPECIAL_CONSIDERATIONS"], inplace=True)

In [55]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ["T9", "T13", "T12", "T2", "T25", "T14", "T29", "T15", "T17"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df_2['APPLICATION_TYPE'] = application_df_2['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df_2['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [56]:
# Choose a smaller cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
counts = application_df_2.groupby("CLASSIFICATION").count()
low_counts = counts.loc[counts["STATUS"] < 95]
low_counts.reset_index(inplace=True)
classifications_to_replace = list(low_counts["CLASSIFICATION"])

# print(classifications_to_replace)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df_2['CLASSIFICATION'] = application_df_2['CLASSIFICATION'].replace(cls,"Other")
    
# # Check to make sure binning was successful
application_df_2['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      574
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
Name: CLASSIFICATION, dtype: int64

In [57]:
# Check data types
application_df_2.dtypes

APPLICATION_TYPE    object
AFFILIATION         object
CLASSIFICATION      object
USE_CASE            object
ORGANIZATION        object
STATUS               int64
INCOME_AMT          object
ASK_AMT              int64
IS_SUCCESSFUL        int64
dtype: object

In [58]:
# Convert categorical data to numeric with `pd.get_dummies`
for column in ['APPLICATION_TYPE', 'CLASSIFICATION', "AFFILIATION",\
              'USE_CASE', 'ORGANIZATION', 'INCOME_AMT']:
        dummies = pd.get_dummies(application_df_2[column])
        application_df_2[dummies.columns] = dummies
        
application_df_2.drop(columns=['APPLICATION_TYPE', 'CLASSIFICATION', "AFFILIATION",\
              'USE_CASE', 'ORGANIZATION', 'INCOME_AMT'], inplace=True)

application_df_2

,STATUS,ASK_AMT,IS_SUCCESSFUL,Other,T10,T19,T3,T4,T5,T6,...,Trust,0,1-9999,10000-24999,100000-499999,10M-50M,1M-5M,25000-99999,50M+,5M-10M
0,1,5000,1,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,0,1,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,1,142590,1,0,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0


In [59]:
# Split our preprocessed data into our features and target arrays
X = application_df_2.drop(columns=["IS_SUCCESSFUL"])
y = application_df_2["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [60]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [61]:
# Get number of input dimensions
X_train.shape

(25724, 45)

## Compile, Train and Evaluate Model - with three hidden layers & more neurons

In [62]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn_model_2 = tf.keras.models.Sequential()

# First hidden layer
nn_model_2.add(tf.keras.layers.Dense(units=160, activation="relu", input_dim=45))

# Second hidden layer
nn_model_2.add(tf.keras.layers.Dense(units=90, activation="relu"))

# Third hidden layer
nn_model_2.add(tf.keras.layers.Dense(units=40, activation="relu"))

# Output layer
nn_model_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model_2.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 160)               7360      
                                                                 
 dense_22 (Dense)            (None, 90)                14490     
                                                                 
 dense_23 (Dense)            (None, 40)                3640      
                                                                 
 dense_24 (Dense)            (None, 1)                 41        
                                                                 
Total params: 25531 (99.73 KB)
Trainable params: 25531 (99.73 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [63]:
# Compile the model
nn_model_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [64]:
# Train the model
fit_model = nn_model_2.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5646 - accuracy: 0.7227
Epoch 2/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5528 - accuracy: 0.7306
Epoch 3/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5501 - accuracy: 0.7300
Epoch 4/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5476 - accuracy: 0.7318
Epoch 5/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5463 - accuracy: 0.7330
Epoch 6/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5450 - accuracy: 0.7342
Epoch 7/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5444 - accuracy: 0.7339
Epoch 8/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5432 - accuracy: 0.7345
Epoch 9/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5421 - accuracy: 0.7359
Epoch 10/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5419 - accura

804/804 [==============================] - 2s 2ms/step - loss: 0.5280 - accuracy: 0.7418
Epoch 82/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5292 - accuracy: 0.7419
Epoch 83/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5284 - accuracy: 0.7420
Epoch 84/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5276 - accuracy: 0.7418
Epoch 85/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5281 - accuracy: 0.7424
Epoch 86/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5282 - accuracy: 0.7427
Epoch 87/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5282 - accuracy: 0.7427
Epoch 88/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5281 - accuracy: 0.7418
Epoch 89/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5280 - accuracy: 0.7424
Epoch 90/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5280 - accuracy: 

In [65]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5946 - accuracy: 0.7304 - 430ms/epoch - 2ms/step
Loss: 0.5945833921432495, Accuracy: 0.7303789854049683


## Re-Preprocessing - include Special Considerations column and increase number of Application Type bins (keep Classification bins increased)

In [66]:
# Read in the original file for reprocessing
application_df_3 = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df_3.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [67]:
# Drop the non-beneficial ID columns, 'EIN', 'NAME'
application_df_3.drop(columns=["EIN", "NAME"], inplace=True)

In [68]:
# Choose a smaller cutoff value and create a list of application types to be replaced
application_types_to_replace = ["T2", "T25", "T14", "T29", "T15", "T17"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df_2['APPLICATION_TYPE'] = application_df_3['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure3binning was successful
application_df_3['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [69]:
# Choose a smaller cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
counts = application_df_3.groupby("CLASSIFICATION").count()
low_counts = counts.loc[counts["STATUS"] < 95]
low_counts.reset_index(inplace=True)
classifications_to_replace = list(low_counts["CLASSIFICATION"])

# print(classifications_to_replace)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df_3['CLASSIFICATION'] = application_df_3['CLASSIFICATION'].replace(cls,"Other")
    
# # Check to make sure binning was successful
application_df_3['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      574
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
Name: CLASSIFICATION, dtype: int64

In [70]:
# Check data types
application_df_3.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [72]:
# Convert categorical data to numeric with `pd.get_dummies`
for column in ['APPLICATION_TYPE', 'CLASSIFICATION', "AFFILIATION",\
              'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']:
        dummies = pd.get_dummies(application_df_3[column])
        application_df_3[dummies.columns] = dummies
        
application_df_3.drop(columns=['APPLICATION_TYPE', 'CLASSIFICATION', "AFFILIATION",\
              'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'], inplace=True)

application_df_3

,STATUS,ASK_AMT,IS_SUCCESSFUL,T10,T12,T13,T14,T15,T17,T19,...,1-9999,10000-24999,100000-499999,10M-50M,1M-5M,25000-99999,50M+,5M-10M,N,Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [73]:
# Split our preprocessed data into our features and target arrays
X = application_df_3.drop(columns=["IS_SUCCESSFUL"])
y = application_df_3["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [74]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [75]:
# Get number of input dimensions
X_train.shape

(25724, 56)

## Compile, Train and Evaluate Model - with three hidden layers, more neurons, & more epochs

In [76]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn_model_3 = tf.keras.models.Sequential()

# First hidden layer
nn_model_3.add(tf.keras.layers.Dense(units=200, activation="relu", input_dim=56))

# Second hidden layer
nn_model_3.add(tf.keras.layers.Dense(units=100, activation="relu"))

# Third hidden layer
nn_model_3.add(tf.keras.layers.Dense(units=50, activation="relu"))

# Output layer
nn_model_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model_3.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            (None, 200)               11400     
                                                                 
 dense_26 (Dense)            (None, 100)               20100     
                                                                 
 dense_27 (Dense)            (None, 50)                5050      
                                                                 
 dense_28 (Dense)            (None, 1)                 51        
                                                                 
Total params: 36601 (142.97 KB)
Trainable params: 36601 (142.97 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [77]:
# Compile the model
nn_model_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [78]:
# Train the model
fit_model = nn_model_3.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5652 - accuracy: 0.7234
Epoch 2/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5531 - accuracy: 0.7297
Epoch 3/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5499 - accuracy: 0.7322
Epoch 4/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5480 - accuracy: 0.7329
Epoch 5/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5465 - accuracy: 0.7338
Epoch 6/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5451 - accuracy: 0.7342
Epoch 7/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5441 - accuracy: 0.7349
Epoch 8/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5421 - accuracy: 0.7351
Epoch 9/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5424 - accuracy: 0.7357
Epoch 10/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5424 - accura

804/804 [==============================] - 2s 2ms/step - loss: 0.5279 - accuracy: 0.7421
Epoch 82/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5282 - accuracy: 0.7420
Epoch 83/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5285 - accuracy: 0.7414
Epoch 84/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5281 - accuracy: 0.7426
Epoch 85/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5277 - accuracy: 0.7425
Epoch 86/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5281 - accuracy: 0.7419
Epoch 87/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5275 - accuracy: 0.7418
Epoch 88/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5275 - accuracy: 0.7420
Epoch 89/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5276 - accuracy: 0.7424
Epoch 90/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5271 - accuracy: 

804/804 [==============================] - 2s 3ms/step - loss: 0.5252 - accuracy: 0.7430
Epoch 161/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5257 - accuracy: 0.7432
Epoch 162/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5248 - accuracy: 0.7437
Epoch 163/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5282 - accuracy: 0.7431
Epoch 164/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5248 - accuracy: 0.7430
Epoch 165/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5251 - accuracy: 0.7432
Epoch 166/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5253 - accuracy: 0.7429
Epoch 167/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5250 - accuracy: 0.7427
Epoch 168/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5268 - accuracy: 0.7430
Epoch 169/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5254 - a

In [80]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6014 - accuracy: 0.7318 - 739ms/epoch - 3ms/step
Loss: 0.601396381855011, Accuracy: 0.7317784428596497


In [81]:
# Export our model to HDF5 file
nn_model_3.save("AlphabetSoupCharity_Optimisation.h5")

C:\Users\AnnaKhayrullina\anaconda3\envs\PythonData\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
